https://medium.com/@rushing_andrei/function-calling-with-open-source-llms-594aa5b3a304

In [18]:
def afunction(arg1:int = 0, arg2:str = "hello", **kwargs)->int:
    ''' this is a function definition
        arg1 (int): an exemplary yet modest argument
        arg2 (str): another nice argument
        **kwargs : the rest of the rabble 

        returns arg1 incremented by one
    '''
    return(arg + 1)

In [22]:
print(afunction.__annotations__)

print(afunction.__name__)

print(afunction.__doc__)

{'arg1': <class 'int'>, 'arg2': <class 'str'>, 'return': <class 'int'>}
afunction
 this is a function definition
        arg1 (int): an exemplary yet modest argument
        arg2 (str): another nice argument
        **kwargs : the rest of the rabble 

        returns arg1 incremented by one
    


In [25]:
import inspect
print(inspect.signature(afunction))

(arg1: int = 0, arg2: str = 'hello', **kwargs) -> int


In [26]:
def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The sum of the two numbers.
    """
    return a + b

In [27]:
print(inspect.signature(add_two_numbers))

(a: int, b: int) -> int


In [30]:
import requests

url = "https://v2.jokeapi.dev/joke/Any?safe-mode&type=twopart"

response = requests.get(url)
print(response.json())

{'error': False, 'category': 'Pun', 'type': 'twopart', 'setup': 'Why should you never talk to pi?', 'delivery': 'Because it will go on forever.', 'flags': {'nsfw': False, 'religious': False, 'political': False, 'racist': False, 'sexist': False, 'explicit': False}, 'safe': True, 'id': 318, 'lang': 'en'}


In [31]:
print(response.json()["setup"])


Why should you never talk to pi?


In [13]:
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [15]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='llama-3.2-90b-vision-preview')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002851FE19E10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002851FE12E90>, model_name='llama-3.2-90b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
from langchain_groq.chat_models import ChatGroq
from langchain.schema import HumanMessage, AIMessage, SystemMessage
import re

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Define the addition function
def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The sum of the two numbers.
    """
    return a + b

# Define the multiplication function
def multiply_two_numbers(a: int, b: int) -> int:
    """
    Multiply two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The product of the two numbers.
    """
    return a * b


# Function to process the user's prompt
def process_prompt(prompt: str, stored_number: int = None, operation: str = None):
    # Extract numbers from the prompt
    numbers = re.findall(r'-?\d+', prompt)  # Handle negative numbers
    numbers = [int(num) for num in numbers]

    # Determine the operation based on the prompt
    if "add" in prompt.lower() or "sum" in prompt.lower():
        operation = "add"
    elif "multiply" in prompt.lower() or "product" in prompt.lower():
        operation = "multiply"

    if stored_number is not None:
        # If a number is already stored, use it as the first number
        if len(numbers) == 1:
            # Perform the operation with the stored number and the new number
            if operation == "add":
                result = add_two_numbers(stored_number, numbers[0])
                return f"The sum of {stored_number} and {numbers[0]} is {result}.", None, None
            elif operation == "multiply":
                result = multiply_two_numbers(stored_number, numbers[0])
                return f"The product of {stored_number} and {numbers[0]} is {result}.", None, None
            else:
                return "Please specify the operation (add or multiply).", stored_number, operation
        else:
            # If no new number is provided, prompt for the second number
            return f"You provided the number {stored_number}. Please provide the second number.", stored_number, operation
    else:
        # If no number is stored, check the current input
        if len(numbers) == 2:
            # If two numbers are found, perform the operation
            if operation == "add":
                result = add_two_numbers(numbers[0], numbers[1])
                return f"The sum of {numbers[0]} and {numbers[1]} is {result}.", None, None
            elif operation == "multiply":
                result = multiply_two_numbers(numbers[0], numbers[1])
                return f"The product of {numbers[0]} and {numbers[1]} is {result}.", None, None
            else:
                return "Please specify the operation (add or multiply).", None, operation
        elif len(numbers) == 1:
            # If one number is found, store it and prompt for the second number
            return f"You provided the number {numbers[0]}. Please provide the second number.", numbers[0], operation
        else:
            # If no numbers are found, prompt the user for both numbers
            return "Please provide two numbers to perform the operation.", None, operation

# Main interaction loop
def main():
    # Initial system message
    system_message = SystemMessage(content="You are a helpful assistant that can add or multiply two numbers.")

    # Initialize stored_number and operation
    stored_number = None
    operation = None

    # Get user input
    user_input = input("User: ")
    human_message = HumanMessage(content=user_input)

    # Process the prompt
    response_content, stored_number, operation = process_prompt(user_input, stored_number, operation)
    ai_message = AIMessage(content=response_content)
    
    # Display the AI's response
    print(f"Assistant: {ai_message.content}")

    # If the AI is prompting for more information, get the user's response
    while "Please provide" in ai_message.content:
        user_input = input("User: ")
        human_message = HumanMessage(content=user_input)
        response_content, stored_number, operation = process_prompt(user_input, stored_number, operation)
        ai_message = AIMessage(content=response_content)
        print(f"Assistant: {ai_message.content}")

if __name__ == "__main__":
    main()

Assistant: Please provide two numbers to perform the operation.
Assistant: You provided the number 5. Please provide the second number.
Assistant: The sum of 5 and 6 is 11.


In [11]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Define the addition function as a tool
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

# Define the multiplication function as a tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

# List of tools
tools = [add, multiply]

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# Main interaction loop
def main():
    # Initial system message
    system_message = SystemMessage(content="You are a helpful assistant that can add or multiply two numbers.")

    # Conversation history
    messages = [system_message]

    while True:
        # Get user input
        user_input = input("User: ")
        messages.append(HumanMessage(content=user_input))

        # Invoke the LLM with tools
        ai_msg = llm_with_tools.invoke(messages)

        # Print the tool calls (if any)
        if hasattr(ai_msg, "tool_calls") and ai_msg.tool_calls:
            print("Tool Calls:", ai_msg.tool_calls)

            # Invoke the tools and append the results to the conversation
            for tool_call in ai_msg.tool_calls:
                selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
                tool_result = selected_tool.invoke(tool_call["args"])
                tool_msg = ToolMessage(content=str(tool_result), name=tool_call["name"], tool_call_id=tool_call["id"])
                messages.append(tool_msg)

            # Invoke the LLM again with the tool results to generate a final response
            final_response = llm_with_tools.invoke(messages)
            print(f"Assistant: {final_response.content}")
            break  # Exit the loop after processing the tool calls
        else:
            # If no tool calls, just display the AI's response
            print(f"Assistant: {ai_msg.content}")
            break  # Exit the loop if no tool calls are needed

if __name__ == "__main__":
    main()

Tool Calls: [{'name': 'multiply', 'args': {'a': 5, 'b': 6}, 'id': 'call_fhvw', 'type': 'tool_call'}]
Assistant: 


In [1]:
import requests
import json

def fetch_movies():
    """
    Fetches 100 movies with IMDB ratings from Dummy API.
    
    Returns:
        dict: A dictionary containing the movies data.
    """
    response = requests.get('https://dummyapi.online/api/movies')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

In [2]:
fetch_movies()

[{'id': 1,
  'movie': 'The Shawshank Redemption',
  'rating': 9.2,
  'image': 'images/shawshank.jpg',
  'imdb_url': 'https://www.imdb.com/title/tt0111161/'},
 {'id': 2,
  'movie': 'The Godfather',
  'rating': 9.2,
  'image': 'images/godfather.jpg',
  'imdb_url': 'https://www.imdb.com/title/tt0068646/'},
 {'id': 3,
  'movie': 'The Dark Knight',
  'rating': 9,
  'image': 'images/dark_knight.jpg',
  'imdb_url': 'https://www.imdb.com/title/tt0468569/'},
 {'id': 4,
  'movie': 'Pulp Fiction',
  'rating': 8.9,
  'image': 'images/pulp_fiction.jpg',
  'imdb_url': 'https://www.imdb.com/title/tt0110912/'},
 {'id': 5,
  'movie': 'The Lord of the Rings: The Return of the King',
  'rating': 9,
  'image': 'images/lotr_return_king.jpg',
  'imdb_url': 'https://www.imdb.com/title/tt0167260/'},
 {'id': 6,
  'movie': 'The Good, the Bad and the Ugly',
  'rating': 8.8,
  'image': 'images/good_bad_ugly.jpg',
  'imdb_url': 'https://www.imdb.com/title/tt0060196/'},
 {'id': 7,
  'movie': 'Fight Club',
  'rating

### Movies API:
1. What is the rating of The ShawWhat is the genre of Inception?shank Redemption?
2. What is the genre of Inception?
3. How long is the runtime of The Godfather?
4. Who are the main actors in The Dark Knight?
5. Show me the release year of Pulp Fiction.


In [38]:
from langchain_groq.chat_models import ChatGroq
from langchain.schema import HumanMessage, AIMessage, SystemMessage
import requests
import json

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Function to fetch movies data from the Dummy API
def fetch_movies():
    """
    Fetches 100 movies with IMDB ratings from Dummy API.
    
    Returns:
        dict: A dictionary containing the movies data.
    """
    response = requests.get('https://dummyapi.online/api/movies')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to process the user's prompt
def process_prompt(prompt: str, movies_data: dict):
    # Create a system message with the movies data
    system_message = SystemMessage(content=f"Movies Data: {movies_data}")
    
    # Create a human message with the user's prompt
    human_message = HumanMessage(content=prompt)
    print(human_message)
    # Generate a response using the LLM
    ai_message = llm([system_message, human_message])
    print(ai_message)
    return ai_message.content

# Main interaction loop
def main():
    # Fetch movies data
    movies_data = fetch_movies()
    if movies_data:
        # Get user input
        user_input = input("User: ")

        # Process the prompt
        response_content = process_prompt(user_input, movies_data)

        # Display the AI's response
        print(f"Assistant: {response_content}")
    else:
        print("Failed to fetch movies data.")

if __name__ == "__main__":
    main()


content='what is the release year of Pulp Fiction' additional_kwargs={} response_metadata={}
content='The release year of Pulp Fiction is 1994.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 5436, 'total_tokens': 5449, 'completion_time': 0.052481239, 'prompt_time': 0.654305051, 'queue_time': 0.10779112499999999, 'total_time': 0.70678629}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'stop', 'logprobs': None} id='run-32f24afc-3f11-4d12-8a0e-5ebb71518ddb-0' usage_metadata={'input_tokens': 5436, 'output_tokens': 13, 'total_tokens': 5449}
Assistant: The release year of Pulp Fiction is 1994.



### Blogposts API:
1. How do I write a blog post about AI in healthcare?
2. Can you provide an example introduction for a blog post on remote work productivity?
3. What are some popular blog post titles on technology trends?
4. How do I structure a blog post on sustainable living practices?
5. What are some common topics in blog posts about travel?


In [19]:
def fetch_products():
    """
    Fetches products data from Dummy API.
    
    Returns:
        dict: A dictionary containing the products data.
    """
    response = requests.get('https://dummyapi.online/api/products')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None
products = fetch_products()

## products

### Products API:
1. What are the color options available for the iPhone 12 Pro?
2. What is the base price of the MacBook Pro 14-inch?
3. How many stock units are available for the iPad Pro 12.9-inch (5th generation)?
4. What is the CPU specification of the iPhone 14 Pro Max?
5. Show me the camera specifications of the iPhone 12 Pro.

In [34]:
print(len(products))
print(products[23])
print(products[23]['productCategory'])

24
{'id': 24, 'productCategory': 'Laptop', 'name': 'MacBook Air (M1, 2021)', 'brand': 'Apple', 'description': 'The MacBook Air (M1, 2021) redefines performance and portability with the Apple M1 chip, a 13-inch Retina display, and silent fanless design.', 'basePrice': 1099, 'inStock': True, 'stock': 20, 'featuredImage': 'https://placehold.co/800x600?text=MacBook+Air+(M1,+2021)', 'thumbnailImage': 'https://placehold.co/400x300?text=MacBook+Air+(M1,+2021)', 'storageOptions': ['256GB', '512GB', '1TB'], 'colorOptions': ['Gold', 'Silver', 'Space Gray'], 'display': '13-inch Retina display', 'CPU': 'Apple M1 chip', 'GPU': 'Apple M1 chip'}
Laptop


In [17]:
from langchain_groq.chat_models import ChatGroq
from langchain.schema import HumanMessage, SystemMessage
import requests
import json

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Function to fetch products data from the Dummy API
def fetch_products():
    """
    Fetches products data from Dummy API.
    
    Returns:
        dict: A dictionary containing the products data.
    """
    response = requests.get('https://dummyapi.online/api/products')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to process the user's prompt
def process_prompt(prompt: str, products_data: dict):
    # Create a system message with the products data
    system_message = SystemMessage(content=f"Products Data: {products_data}")

    # Create a human message with the user's prompt
    human_message = HumanMessage(content=prompt)

    # Generate a response using the LLM
    ai_message = llm([system_message, human_message])

    return ai_message.content

# Main interaction loop
def main():
    # Fetch products data
    products_data = fetch_products()
    if products_data:
        # Get user input
        user_input = input("User: ")

        # Process the prompt
        response_content = process_prompt(user_input, products_data)

        # Display the AI's response
        print(f"Assistant: {response_content}")
    else:
        print("Failed to fetch products data.")

if __name__ == "__main__":
    main()


Assistant: The color options available for the iPhone 12 Pro are:

1. Graphite
2. Silver
3. Gold
4. Pacific Blue


### pokemon


### Pokémon API:
1. How many hitpoints does Pikachu have?
2. What are the abilities of Bulbasaur?
3. In which location can I find Eevee?
4. What are the evolutions of Charmander?
5. Show me the image URL of Snorlax.

In [37]:
from langchain_groq.chat_models import ChatGroq
from langchain.schema import HumanMessage, SystemMessage
import requests
import json

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Function to fetch pokemon data from the Dummy API
def fetch_pokemon():
    """
    Fetches pokemon data from Dummy API.
    
    Returns:
        dict: A dictionary containing the pokemon data.
    """
    response = requests.get('https://dummyapi.online/api/pokemon')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None



# Function to process the user's prompt
def process_prompt(prompt: str, pokemon_data: dict):
    # Create a system message with the pokemon data
    system_message = SystemMessage(content=f"Pokemon Data: {pokemon_data}")

    # Create a human message with the user's prompt
    human_message = HumanMessage(content=prompt)

    # Generate a response using the LLM
    ai_message = llm([system_message, human_message])

    return ai_message.content

# Main interaction loop
def main():
    # Fetch pokemon data
    pokemon_data = fetch_pokemon()
    if pokemon_data:
        # Get user input
        user_input = input("User: ")

        # Process the prompt
        response_content = process_prompt(user_input, pokemon_data)

        # Display the AI's response
        print(f"Assistant: {response_content}")
    else:
        print("Failed to fetch pokemon data.")

if __name__ == "__main__":
    main()

Assistant: The evolutions of Charmander are:

1. Charmeleon
2. Charizard


## users

### Users API:
1. What is the email address of John Doe?
2. Where does Jane Doe live?
3. What is the username of Jim Smith?
4. Show me the address details of Sara Wilson.
5. What city does Bob Johnson reside in?

In [39]:
from langchain_groq.chat_models import ChatGroq
from langchain.schema import HumanMessage, SystemMessage
import requests
import json

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Function to fetch users data from the Dummy API
def fetch_users():
    """
    Fetches users data from Dummy API.
    
    Returns:
        dict: A dictionary containing the users data.
    """
    response = requests.get('https://dummyapi.online/api/users')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to process the user's prompt
def process_prompt(prompt: str, users_data: dict):
    # Create a system message with the users data
    system_message = SystemMessage(content=f"Users Data: {users_data}")

    # Create a human message with the user's prompt
    human_message = HumanMessage(content=prompt)

    # Generate a response using the LLM
    ai_message = llm([system_message, human_message])

    return ai_message.content

# Main interaction loop
def main():
    # Fetch users data
    users_data = fetch_users()
    if users_data:
        # Get user input
        user_input = input("User: ")

        # Process the prompt
        response_content = process_prompt(user_input, users_data)

        # Display the AI's response
        print(f"Assistant: {response_content}")
    else:
        print("Failed to fetch users data.")

if __name__ == "__main__":
    main()


Assistant: Jane Doe lives at 456 Market St, Los Angeles, CA 90001.


In [13]:
from langchain_groq.chat_models import ChatGroq
from langchain.schema import HumanMessage, SystemMessage
import requests
import json

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Function to classify the user's query using the LLM
def classify_query(query: str) -> str:
    """
    Classifies the user's query to determine which API endpoint to fetch data from.
    
    Args:
        query (str): The user's input query.
    
    Returns:
        str: The API endpoint to fetch data from.
    """
    if not query or not query.strip():
        return None

    # Define the system message to guide the LLM
    system_message = SystemMessage(content="""
    You are a helpful assistant that classifies user queries into one of the following categories:
    - products: If the query is related to products, items, or goods.
    - pokemon: If the query is related to Pokémon, creatures, or games.
    - users: If the query is related to users, people, or profiles.
    - movies: If the query is related to movies, films, or IMDB ratings.
    
    Return only the category name (e.g., "products" ,"pokemon, "users", "movies etc.) based on the user's query.
    """)

    # Create a human message with the user's query
    human_message = HumanMessage(content=query)

    try:
        # Generate a response using the LLM
        ai_message = llm.invoke([system_message, human_message])

        # Extract the category from the LLM's response
        category = ai_message.content.strip().lower()

        # Validate the category
        valid_categories = ["products", "pokemon", "users", "movies"]
        if category in valid_categories:
            return category
        else:
            return None
    except Exception as e:
        print(f"Error classifying query: {e}")
        return None

# Function to fetch data from the Dummy API
def fetch_data(api_endpoint: str):
    """
    Fetches data from the specified Dummy API endpoint.
    
    Args:
        api_endpoint (str): The API endpoint to fetch data from.
    
    Returns:
        dict: A dictionary containing the fetched data.
    """
    if not api_endpoint:
        return None

    url = f"https://dummyapi.online/api/{api_endpoint}"
    try:
        response = requests.get(url, timeout=10)  # Add a timeout to avoid hanging
        if response.status_code == 200:
            return json.loads(response.text)
        else:
            print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Network error while fetching data from {url}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Invalid JSON data received from {url}: {e}")
        return None

# Function to process the user's prompt
def process_prompt(prompt: str, data: dict):
    """
    Processes the user's prompt using the fetched data.
    
    Args:
        prompt (str): The user's input prompt.
        data (dict): The fetched data from the API.
    
    Returns:
        str: The AI's response to the user's prompt.
    """
    if not prompt or not data:
        return "Sorry, I couldn't process your request due to missing or invalid data."

    try:
        # Create a system message with the fetched data
        system_message = SystemMessage(content=f"Data: {data}")

        # Create a human message with the user's prompt
        human_message = HumanMessage(content=prompt)

        # Generate a response using the LLM
        ai_message = llm.invoke([system_message, human_message])

        return ai_message.content
    except Exception as e:
        print(f"Error processing prompt: {e}")
        return "Sorry, I encountered an error while processing your request."

# Main function to run the program once
def main():
    # Get user input
    user_input = input("User: ").strip()

    if not user_input:
        print("Please provide a valid query.")
        return

    # Classify the query to determine which API to fetch data from
    api_endpoint = classify_query(user_input)
    
    if not api_endpoint:
        print("Sorry, I couldn't determine which data to fetch based on your query.")
        return

    # Fetch data from the appropriate API
    data = fetch_data(api_endpoint)
    
    if not data:
        print(f"Failed to fetch data from the {api_endpoint} API.")
        return

    # Process the prompt using the fetched data
    response_content = process_prompt(user_input, data)
    
    # Display the AI's response
    print(f"Assistant: {response_content}")

if __name__ == "__main__":
    main()

Assistant: According to the data, the location of Farfetch'd is Kanto region, Route 12.


In [12]:
from langchain_groq.chat_models import ChatGroq
from langchain.schema import HumanMessage, SystemMessage
import requests
import json

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Function to classify the user's query using the LLM
def classify_query(query: str) -> str:
    """
    Classifies the user's query to determine which API endpoint to fetch data from.
    
    Args:
        query (str): The user's input query.
    
    Returns:
        str: The API endpoint to fetch data from.
    """
    # Define the system message to guide the LLM
    system_message = SystemMessage(content="""
    You are a helpful assistant that classifies user queries into one of the following categories:
    - products: If the query is related to products, items, or goods.
    - pokemon: If the query is related to Pokémon, creatures, or games.
    - users: If the query is related to users, people, or profiles.
    - movies: If the query is related to movies, films, or IMDB ratings.
    
    Return only the category name (e.g., "products", "pokemon", etc.) based on the user's query.
    """)

    # Create a human message with the user's query
    human_message = HumanMessage(content=query)

    # Generate a response using the LLM
    ai_message = llm.invoke([system_message, human_message])

    # Extract the category from the LLM's response
    category = ai_message.content.strip().lower()

    # Validate the category
    valid_categories = ["products", "pokemon", "users", "movies"]
    if category in valid_categories:
        return category
    else:
        return None

# Function to fetch products data from the Dummy API
def fetch_products():
    """
    Fetches products data from Dummy API.
    
    Returns:
        dict: A dictionary containing the products data.
    """
    response = requests.get('https://dummyapi.online/api/products')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to fetch pokemon data from the Dummy API
def fetch_pokemon():
    """
    Fetches pokemon data from Dummy API.
    
    Returns:
        dict: A dictionary containing the pokemon data.
    """
    response = requests.get('https://dummyapi.online/api/pokemon')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to fetch users data from the Dummy API
def fetch_users():
    """
    Fetches users data from Dummy API.
    
    Returns:
        dict: A dictionary containing the users data.
    """
    response = requests.get('https://dummyapi.online/api/users')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to fetch movies data from the Dummy API
def fetch_movies():
    """
    Fetches 100 movies with IMDB ratings from Dummy API.
    
    Returns:
        dict: A dictionary containing the movies data.
    """
    response = requests.get('https://dummyapi.online/api/movies')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to fetch data from the appropriate API
def fetch_data(api_endpoint: str):
    """
    Fetches data from the specified Dummy API endpoint.
    
    Args:
        api_endpoint (str): The API endpoint to fetch data from.
    
    Returns:
        dict: A dictionary containing the fetched data.
    """
    if api_endpoint == "products":
        return fetch_products()
    elif api_endpoint == "pokemon":
        return fetch_pokemon()
    elif api_endpoint == "users":
        return fetch_users()
    elif api_endpoint == "movies":
        return fetch_movies()
    else:
        return None

# Function to process the user's prompt
def process_prompt(prompt: str, data: dict):
    """
    Processes the user's prompt using the fetched data.
    
    Args:
        prompt (str): The user's input prompt.
        data (dict): The fetched data from the API.
    
    Returns:
        str: The AI's response to the user's prompt.
    """
    # Create a system message with the fetched data
    system_message = SystemMessage(content=f"Data: {data}")

    # Create a human message with the user's prompt
    human_message = HumanMessage(content=prompt)

    # Generate a response using the LLM
    ai_message = llm.invoke([system_message, human_message])

    return ai_message.content

# Main function to run the program once
def main():
    # Get user input
    user_input = input("User: ")

    # Classify the query to determine which API to fetch data from
    api_endpoint = classify_query(user_input)
    
    if api_endpoint:
        # Fetch data from the appropriate API
        data = fetch_data(api_endpoint)
        
        if data:
            # Process the prompt using the fetched data
            response_content = process_prompt(user_input, data)
            
            # Display the AI's response
            print(f"Assistant: {response_content}")
        else:
            print(f"Failed to fetch data from the {api_endpoint} API.")
    else:
        print("Sorry, I couldn't determine which data to fetch based on your query.")

if __name__ == "__main__":
    main()

Assistant: According to the provided data, the rating of "The Dark Knight" movie is 9.


### Movies API:
1. What is the rating of The ShawWhat is the genre of Inception?shank Redemption?
2. What is the genre of Inception?
3. How long is the runtime of The Godfather?
4. Who are the main actors in The Dark Knight?
5. Show me the release year of Pulp Fiction.



### Pokémon API:
1. How many hitpoints does Pikachu have?
2. What are the abilities of Bulbasaur?
3. In which location can I find Eevee?
4. What are the evolutions of Charmander?
5. Show me the image URL of Snorlax.

### Users API:
1. What is the email address of John Doe?
2. Where does Jane Doe live?
3. What is the username of Jim Smith?
4. Show me the address details of Sara Wilson.
5. What city does Bob Johnson reside in?

In [6]:
from langchain_groq.chat_models import ChatGroq
from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
import requests
import json

# Initialize the ChatGroq LLM
llm = ChatGroq(model='llama-3.2-90b-vision-preview')

# Function to classify the user's query using the LLM
def classify_query(query: str) -> str:
    """
    Classifies the user's query to determine which API endpoint to fetch data from.
    
    Args:
        query (str): The user's input query.
    
    Returns:
        str: The API endpoint to fetch data from.
    """
    # Define the system message to guide the LLM
    system_message = SystemMessage(content="""
    You are a helpful assistant that classifies user queries into one of the following categories:
    - products: If the query is related to products, items, or goods.
    - pokemon: If the query is related to Pokémon, creatures, or games.
    - users: If the query is related to users, people, or profiles.
    - movies: If the query is related to movies, films, or IMDB ratings.
    
    Return only the category name (e.g., "products", "pokemon", etc.) based on the user's query.
    """)

    # Create a human message with the user's query
    human_message = HumanMessage(content=query)

    # Generate a response using the LLM
    ai_message = llm.invoke([system_message, human_message])

    # Extract the category from the LLM's response
    category = ai_message.content.strip().lower()

    # Validate the category
    valid_categories = ["products", "pokemon", "users", "movies"]
    if category in valid_categories:
        return category
    else:
        return None

# Function to fetch products data from the Dummy API
def fetch_products():
    """
    Fetches products data from Dummy API.
    
    Returns:
        dict: A dictionary containing the products data.
    """
    response = requests.get('https://dummyapi.online/api/products')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to fetch pokemon data from the Dummy API
def fetch_pokemon():
    """
    Fetches pokemon data from Dummy API.
    
    Returns:
        dict: A dictionary containing the pokemon data.
    """
    response = requests.get('https://dummyapi.online/api/pokemon')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to fetch users data from the Dummy API
def fetch_users():
    """
    Fetches users data from Dummy API.
    
    Returns:
        dict: A dictionary containing the users data.
    """
    response = requests.get('https://dummyapi.online/api/users')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to fetch movies data from the Dummy API
def fetch_movies():
    """
    Fetches 100 movies with IMDB ratings from Dummy API.
    
    Returns:
        dict: A dictionary containing the movies data.
    """
    response = requests.get('https://dummyapi.online/api/movies')
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Function to fetch data from the appropriate API
def fetch_data(api_endpoint: str):
    """
    Fetches data from the specified Dummy API endpoint.
    
    Args:
        api_endpoint (str): The API endpoint to fetch data from.
    
    Returns:
        dict: A dictionary containing the fetched data.
    """
    if api_endpoint == "products":
        return fetch_products()
    elif api_endpoint == "pokemon":
        return fetch_pokemon()
    elif api_endpoint == "users":
        return fetch_users()
    elif api_endpoint == "movies":
        return fetch_movies()
    else:
        return None

# Define a custom tool for fetching data based on the query
def fetch_data_tool(query: str) -> str:
    """
    Custom tool to fetch data based on the user's query.
    
    Args:
        query (str): The user's input query.
    
    Returns:
        str: The fetched data or an error message.
    """
    api_endpoint = classify_query(query)
    if api_endpoint:
        data = fetch_data(api_endpoint)
        if data:
            return json.dumps(data)
        else:
            return f"Failed to fetch data from the {api_endpoint} API."
    else:
        return "Sorry, I couldn't determine which data to fetch based on your query."

# Create a Tool object for the custom fetch_data_tool
fetch_data_tool = Tool(
    name="FetchData",
    func=fetch_data_tool,
    description="Fetches data from the appropriate API based on the user's query."
)

# Initialize the agent with the custom tool
tools = [fetch_data_tool]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Main function to run the program once
def main():
    # Get user input
    user_input = input("User: ")

    # Run the agent with the user's input
    response = agent.invoke(user_input)

    # Display the AI's response
    print(f"Assistant: {response}")

if __name__ == "__main__":
    main()



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to fetch the top 10 rated movies data from a reliable API.

Action: FetchData
Action Input: "top 10 rated movies"

Observation: [{"id": 1, "movie": "The Shawshank Redemption", "rating": 9.2, "image": "images/shawshank.jpg", "imdb_url": "https://www.imdb.com/title/tt0111161/"}, {"id": 2, "movie": "The Godfather", "rating": 9.2, "image": "images/godfather.jpg", "imdb_url": "https://www.imdb.com/title/tt0068646/"}, {"id": 3, "movie": "The Dark Knight", "rating": 9, "image": "images/dark_knight.jpg", "imdb_url": "https://www.imdb.com/title/tt0468569/"}, {"id": 4, "movie": "Pulp Fiction", "rating": 8.9, "image": "images/pulp_fiction.jpg", "imdb_url": "https://www.imdb.com/title/tt0110912/"}, {"id": 5, "movie": "The Lord of the Rings: The Return of the King", "rating": 9, "image": "images/lotr_return_king.jpg", "imdb_url": "https://www.imdb.com/title/tt0167260/"}, {"id": 6, "movie": "The Good, the Bad and the Ug

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Thought: Since the observation already contains a list of the top 100 rated movies, I can simply select the top 10 movies from the list.
`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [2]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

NameError: name 'llm' is not defined

In [1]:
from langchain_core.messages import HumanMessage


query = 'what is 5 + 5'
messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

NameError: name 'llm_with_tools' is not defined

In [37]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='what is sum of 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8w8s', 'function': {'arguments': '{"a": "5", "b": "0"}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 275, 'total_tokens': 294, 'completion_time': 0.10705526, 'prompt_time': 0.035973769, 'queue_time': 0.057925475000000004, 'total_time': 0.143029029}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cbcb5081-c005-49c5-827c-99588d50a68e-0', tool_calls=[{'name': 'add', 'args': {'a': '5', 'b': '0'}, 'id': 'call_8w8s', 'type': 'tool_call'}], usage_metadata={'input_tokens': 275, 'output_tokens': 19, 'total_tokens': 294}),
 ToolMessage(content='5', name='add', tool_call_id='call_8w8s')]

In [43]:
query = "what is 5 * 6"
llm_with_tools.invoke(messages)


AIMessage(content='The sum of 5 is 5.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 305, 'total_tokens': 315, 'completion_time': 0.036260856, 'prompt_time': 0.045685709, 'queue_time': 0.061160079, 'total_time': 0.081946565}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'stop', 'logprobs': None}, id='run-6077775c-a11c-4f89-a387-8889093c9158-0', usage_metadata={'input_tokens': 305, 'output_tokens': 10, 'total_tokens': 315})

In [17]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

model = OllamaFunctions(
    model="llama3.2", 
    format="json"
    )

model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

response = model.invoke("what is the weather in ?")

print(response)

content="I'm not sure what you mean by '?' Could you please provide more context or clarify which location you'd like to know the weather for?" additional_kwargs={} response_metadata={} id='run-d824947e-937c-46be-b0cf-c3960d3ba53b-0'
